# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value of handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Setup

Parameters:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `digits.digits_prepped`
WHERE splits = 'TRAIN'

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 2149.09query/s]                        


""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

Downloading: 100%|██████████| 14/14 [00:01<00:00, 11.40rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.193844,0.193899,0.2,6039
1,0,1,0.139669,0.140008,0.4,7313
2,0,2,0.084853,0.086195,0.8,7178
3,0,3,0.050086,0.053006,1.6,6941
4,0,4,0.031418,0.034946,3.2,7927
5,0,5,0.020791,0.024225,6.4,9092
6,0,6,0.014930,0.019423,12.8,9887
7,0,7,0.013202,0.017322,6.4,9791
8,0,8,0.011923,0.016646,3.2,8654
9,0,9,0.010946,0.016027,6.4,8486


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.90rows/s]


,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TEST,0.951985,0.951856,0.951031,0.951616,0.195121,0.997704
1,TRAIN,0.989174,0.988915,0.989354,0.989006,0.082043,0.999887


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.12rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,133,0,0,0,0,0,0,0,0,0
1,1,0,142,0,0,0,0,0,0,2,0
2,2,0,0,149,0,0,0,0,0,0,0
3,3,0,0,0,138,0,0,0,0,2,0
4,4,0,0,0,0,149,0,0,0,0,0
5,5,0,1,0,0,0,141,0,0,0,0
6,6,0,1,0,0,0,0,145,0,0,0
7,7,0,0,0,0,0,0,0,143,0,0
8,8,0,3,0,0,0,1,1,0,122,0
9,9,0,0,0,1,0,2,0,0,1,132


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.50rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,45,0,0,0,0,0,0,0,0,0
1,1,0,37,0,0,0,0,0,0,1,0
2,2,0,1,27,0,0,0,0,0,0,0
3,3,0,0,1,41,0,0,0,0,1,0
4,4,0,0,0,0,31,0,0,1,0,0
5,5,0,0,0,0,0,37,1,0,0,2
6,6,0,0,0,0,1,0,34,0,0,0
7,7,0,0,0,0,1,0,0,33,0,2
8,8,0,3,0,0,0,0,0,0,44,0
9,9,0,0,0,0,0,1,0,1,2,40


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Downloading: 100%|██████████| 388/388 [00:01<00:00, 271.79rows/s]


Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,2,2,"[{'label': 2, 'prob': 0.9827713584504537}, {'l...",TEST
1,9,9,"[{'label': 9, 'prob': 0.9852888616793595}, {'l...",TEST
2,9,9,"[{'label': 9, 'prob': 0.581938298229805}, {'la...",TEST
3,3,3,"[{'label': 3, 'prob': 0.8681427781777712}, {'l...",TEST
4,9,9,"[{'label': 9, 'prob': 0.8574231193127023}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 8.163074467996867e-05},
  {'label': 1, 'prob': 9.323421999854328e-05},
  {'label': 2, 'prob': 0.9827713584504537},
  {'label': 3, 'prob': 0.016495303189195055},
  {'label': 4, 'prob': 6.442037815360304e-07},
  {'label': 5, 'prob': 0.00011069510839788402},
  {'label': 6, 'prob': 1.3898464657538618e-06},
  {'label': 7, 'prob': 0.00010966311899547486},
  {'label': 8, 'prob': 2.1601851760482518e-05},
  {'label': 9, 'prob': 0.00031447926627172493}]]

---
## Remove Resources
see notebook "XX - Cleanup"